With a new dataset, only modifying the following shall be sufficient. 
* Import Data
    * directory
    * file name
    * pd.read_csv() if different file type
    * data_name
    * label_name
* Prepare Data
    * In get_train_data function, modify the way binary labels are defined.
    * Number of posts (iterations) in get_word_data based on your interest
* Results
    * Modify the way 'keyword' is used based on your binary labels.

# Import Data

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
directory = '/data1/link10/yelp/'
file_name = 'df1M.tsv'
raw_df = pd.read_csv(directory + file_name, delimiter = '\t')
# remove rows with missing values
df = raw_df.dropna()
df.head()

,review_id,stars,text
0,xQY8N_XvtGbearJ5X4QryQ,2,"As someone who has worked with many museums, I..."
1,UmFMZ8PyXZTY2QcwzsfQYA,1,I am actually horrified this place is still in...
2,LG2ZaYiOgpr2DK_90pYjNw,5,I love Deagan's. I do. I really do. The atmosp...
3,i6g_oA9Yf9Y31qt0wibXpw,1,"Dismal, lukewarm, defrosted-tasting ""TexMex"" g..."
4,6TdNDKywdbjoTkizeMce8A,4,"Oh happy day, finally have a Canes near my cas..."


In [3]:
data_name = 'text'
label_name = 'stars'

In [4]:
print(
    'There are {} data.'.format(df.shape[0]),
    'Labels are: {}'.format(df[label_name].unique()),
    sep = '\n'
    )
df[label_name].value_counts()

There are 1000000 data.
Labels are: [2 1 5 4 3]


5    449091
4    210363
1    156690
3    104973
2     78883
Name: stars, dtype: int64

# Embedding

In [5]:
# !python -m spacy download en_core_web_lg
import numpy as np
import spacy

In [6]:
# # load the language model
# nlp = spacy.load('en_core_web_lg')

In [7]:
# with nlp.disable_pipes():
#     msg_vectors = np.array([nlp(msg.lower()).vector for msg in tqdm(df[data_name])])
# msg_vectors.shape

In [8]:
# yelp_glove_vectors = msg_vectors
# %store yelp_glove_vectors

In [9]:
# embedding takes huge amount of time, use stored result
%store -r yelp_glove_vectors
msg_vectors = yelp_glove_vectors
msg_vectors.shape

(1000000, 300)

# Prepare data

In [10]:
from sklearn.model_selection import train_test_split
def get_train_data(keyword):
    labels = np.array([1 if x > keyword else 0 for x in df[label_name]])
    X_train, X_test, Y_train, Y_test = train_test_split(msg_vectors, labels,
                                                    test_size=0.2, random_state=1)
    return X_train, X_test, Y_train, Y_test

In [11]:
def get_word_data(npl,df):
    word_list = []
    word_vec = []
    word_occr_dict = {}
    with nlp.disable_pipes():
        for i in tqdm(range(1000000)):
            msg = nlp(df.iloc[i][data_name].lower())
            for token in msg:
                if token.text not in word_list:
                    word_list.append(token.text)
                    word_vec.append([token.vector])
                    word_occr_dict[token.text] = 1
                else:
                    word_occr_dict[token.text] += 1     
    word_array = np.concatenate(np.array(word_vec),0)
    word_occr = [word_occr_dict[word] for word in word_list]
    return word_list, word_array, word_occr

# Linear SVC


In [12]:
from sklearn.svm import LinearSVC
def top_words_SVC(X_train, X_test, Y_train, Y_test, word_list, word_array, word_occr): 
# Set dual=False to speed up training, and it's not needed
    svc = LinearSVC(random_state=1, dual=False, max_iter=10000)
    svc.fit(X_train, Y_train)
    accu = svc.score(X_test, Y_test)
    scores = svc.decision_function(word_array)
    word_df = pd.DataFrame({'word':word_list,'scores':scores,'occurrence':word_occr})
#     word_df = word_df[word_df['occurrence']>50]
    word_df = word_df.sort_values('scores',ascending=False)
    return accu, word_df

In [13]:
def get_top_words(keyword, word_list, word_array, word_occr):
    X_train, X_test, Y_train, Y_test = get_train_data(keyword)
    accu, words = top_words_SVC(X_train, X_test, Y_train, Y_test, word_list, word_array, word_occr)
    print(
        'Critical star: {}'.format(keyword),
        'Accuracy: {}'.format(accu),
        sep = '\n'
    )
    return words

# Results

In [14]:
# word_list, word_array, word_occr = get_word_data(nlp,df)

In [15]:
# yelp_glove_word = [word_list, word_array, word_occr]
# %store yelp_glove_word

In [16]:
# use stored result
%store -r yelp_glove_word
word_list = yelp_glove_word[0]
word_array = yelp_glove_word[1]
word_occr = yelp_glove_word[2]

In [ ]:
# glove + linear SVC + filter

In [18]:
top_words = get_top_words(3,word_list, word_array, word_occr)

Critical star: 3
Accuracy: 0.88554


In [30]:
thresh = 1000
pos_words_f = top_words[top_words['occurrence']>thresh]

In [31]:
pos_words_f.head(15)

,word,scores,occurrence
313,amazing,21.063763,130807
1829,knowledgeable,20.322675,16204
3429,fabulous,20.078485,9754
84,wonderful,20.014839,37654
7373,energetic,19.689393,1027
1173,gorgeous,19.609969,5277
7583,skilled,19.536980,1297
3477,adorable,19.428992,2402
4860,personable,19.375933,5623
929,fantastic,18.873775,36577


In [32]:
neg_words_f  = pos_words_f.sort_values('scores',ascending=True)
neg_words_f.head(15)

,word,scores,occurrence
9178,disrespectful,-33.910501,1535
5349,tasteless,-33.825856,3938
214,disgusting,-33.752219,6644
1450,stale,-32.518283,3705
5805,unprofessional,-32.472631,6733
1252,pathetic,-32.363460,1309
787,bland,-31.646482,16126
1367,unacceptable,-29.768927,2800
6959,miserable,-29.343382,1426
9537,soggy,-28.887766,7154
